In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#! git clone https://github.com/Tikquuss/meta_XLM /content/meta_XLM

# **A ce niveau si on a des données deja traitées, on copie juste**

**1) On a pas les données**

In [ ]:
#! cp -R /content/drive/"My Drive"/African_Translator/datasets/YourVersion/csvs /content

In [3]:
#%bookmark HOME "/content/meta_XLM/XLM" 
#%cd -b HOME

In [4]:
#%cd tools
#! git clone https://github.com/moses-smt/mosesdecoder
#! git clone https://github.com/glample/fastBPE && cd fastBPE && g++ -std=c++11 -pthread -O3 fastBPE/main.cc -IfastBPE -o fast

In [5]:
#%cd -b HOME

In [ ]:
#! mkdir data

In [ ]:
"""
%env csv_path=/content
%env output_dir=data
%env data_type=para
%env languages=Bafia,MKPAMAN_AMVOE_Ewondo
! python ../bible.py --csv_path $csv_path --output_dir $output_dir --data_type $data_type --languages $languages
"""

In [ ]:
"""
%env PARA=True          
%env MONO=True       
                   
%env PARA_PATH=data/para/Bafia-MKPAMAN_AMVOE_Ewondo     
%env MONO_PATH=""  
%env SAME_VOCAB=True    
%env nCodes=10000             
%env shuf_n_samples=1000000   
%env threads_for_tokenizer=16 
%env test_size=10             
%env val_size=10              

# tools paths
%env TOKENIZE=tools/tokenizer_our.sh
%env LOWER_REMOVE_ACCENT=tools/lowercase_and_remove_accent.py
%env FASTBPE=tools/fastBPE/fast


%env OUTPATH=data/processed
# create output path
! mkdir -p $OUTPATH

! chmod +x $FASTBPE
! chmod +x ../build_meta_data.sh
! chmod +x tools/mosesdecoder/scripts/tokenizer/*.perl

%env n_samples=-1

%env sub_task=Bafia-MKPAMAN_AMVOE_Ewondo:-1
"""

In [6]:
#! ../build_meta_data.sh $sub_task $n_samples

In [7]:
# save the data for future training
#! mkdir /content/drive/"My Drive"/African_Translator/models/africa/cluster1/data/Bafia_Ewondo
#! rsync -av --progress data/processed /content/drive/"My Drive"/African_Translator/models/africa/cluster1/data/Bafia_Ewondo

**2) On a les données**

In [9]:
#%bookmark HOME "/content/meta_XLM/XLM" 
#%cd -b HOME

In [8]:
#! rsync -av --progress /content/drive/"My Drive"/African_Translator/models/africa/cluster1/data/Bafia_Ewondo/processed  data

In [1]:
%bookmark HOME "/home/jupyter/meta_XLM/XLM" 
%cd -b HOME

(bookmark:HOME) -> /home/jupyter/meta_XLM/XLM
/home/jupyter/meta_XLM/XLM


# **Trainnig**

In [3]:
%env OUTPATH=/home/jupyter/models/africa/cluster1/data/Bafia_Ewondo/processed
%env exp_id=maml
# If you don't have enough RAM or swap memory, leave these three parameters to True, otherwise you may get an error like this when evaluating 
# RuntimeError: copy_if failed to synchronize: cudaErrorAssert: device-side assert triggered
%env remove_long_sentences_train=True
%env remove_long_sentences_valid=True
%env remove_long_sentences_test=True
#--remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test

env: OUTPATH=/home/jupyter/models/africa/cluster1/data/Bafia_Ewondo/processed
env: exp_id=maml
env: remove_long_sentences_train=True
env: remove_long_sentences_valid=True
env: remove_long_sentences_test=True


In [4]:
# le plus grand fichier à tensor_len phrases, on cherche le multiple de "batch_size" le plus proche de ce 
# nombre par valeur supérieur : epoch_size doit etre un multiple non nul de ce nombre (pour ne pas gaspiller) 

def getEpochSize(tensor_len, batch_size):
    i = tensor_len
    while True :
        if i%batch_size == 0 :
            return i//batch_size
        i = i + 1

import io

def n_lines(file_path):
    return len(io.open(file_path, encoding='UTF-8').read().split('\n'))

In [5]:
tensor_len = n_lines(file_path = "/home/jupyter/models/africa/cluster1/data/Bafia_Ewondo/processed/Bafia-MKPAMAN_AMVOE_Ewondo.Bafia.train")
print("tensor_len = " + str(tensor_len))

tensor_len = 6358


In [6]:
%env epoch_size=6358

%env lgs=Bafia-MKPAMAN_AMVOE_Ewondo

# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

# stopping criterion (if criterion does not improve 10 times)
%env stopping_criterion=_valid_mlm_ppl,10
%env eval_bleu false

env: epoch_size=6358
env: lgs=Bafia-MKPAMAN_AMVOE_Ewondo
env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: stopping_criterion=_valid_mlm_ppl,10
env: eval_bleu=false


In [16]:
#! rsync -av --progress /content/drive/"My Drive"/African_Translator/models/africa/cluster1/mlm_tlm_BafiaEwondo  /content/meta_XLM/XLM/dumped 

In [7]:
%env mlm_steps=Bafia,MKPAMAN_AMVOE_Ewondo,Bafia-MKPAMAN_AMVOE_Ewondo
%env batch_size=16
%env max_epoch=100
%env dump_path=/home/jupyter/models/africa/cluster1
! python train.py --exp_name mlm_tlm_BafiaEwondo --exp_id $exp_id --dump_path $dump_path --data_path $OUTPATH --lgs $lgs --clm_steps '' --mlm_steps $mlm_steps --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --batch_size $batch_size --bptt 256 --optimizer adam,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --validation_metrics _valid_mlm_ppl --stopping_criterion $stopping_criterion --eval_bleu $eval_bleu --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

env: mlm_steps=Bafia,MKPAMAN_AMVOE_Ewondo,Bafia-MKPAMAN_AMVOE_Ewondo
env: batch_size=16
env: max_epoch=100
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm-bis-vm
INFO - 05/21/20 10:09:14 - 0:00:00 - ============ Initialized logger ============
INFO - 05/21/20 10:09:14 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: []
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 16
                                     beam_size: 1
                                     bptt: 256


INFO - 05/21/20 10:09:14 - 0:00:00 - 275240 words (9128 unique) in 6357 sentences. 0 unknown words (0 unique) covering 0.00% of the data.

INFO - 05/21/20 10:09:14 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluster1/data/Bafia_Ewondo/processed/valid.Bafia.pth ...
INFO - 05/21/20 10:09:14 - 0:00:00 - 35112 words (9128 unique) in 794 sentences. 26 unknown words (23 unique) covering 0.07% of the data.

INFO - 05/21/20 10:09:14 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluster1/data/Bafia_Ewondo/processed/test.Bafia.pth ...
INFO - 05/21/20 10:09:14 - 0:00:00 - 34415 words (9128 unique) in 794 sentences. 28 unknown words (24 unique) covering 0.08% of the data.

INFO - 05/21/20 10:09:14 - 0:00:00 - ============ Monolingual data (MKPAMAN_AMVOE_Ewondo)
INFO - 05/21/20 10:09:14 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluster1/data/Bafia_Ewondo/processed/train.MKPAMAN_AMVOE_Ewondo.pth ...
INFO - 05/21/20 10:09:14 - 0:00:00 - 150775 words (9128 uni

INFO - 05/21/20 10:09:19 - 0:00:05 - Found 0 memories.
INFO - 05/21/20 10:09:19 - 0:00:05 - Found 6 FFN.
INFO - 05/21/20 10:09:19 - 0:00:05 - Found 102 parameters in model.
INFO - 05/21/20 10:09:19 - 0:00:05 - Optimizers: model
INFO - 05/21/20 10:09:19 - 0:00:06 - ============ Starting epoch 0 ... ============
INFO - 05/21/20 10:09:19 - 0:00:06 - Creating new training data iterator (pred,Bafia) ...
INFO - 05/21/20 10:09:21 - 0:00:07 - Creating new training data iterator (pred,MKPAMAN_AMVOE_Ewondo) ...
INFO - 05/21/20 10:09:22 - 0:00:08 - Creating new training data iterator (pred,Bafia,MKPAMAN_AMVOE_Ewondo) ...
INFO - 05/21/20 10:09:30 - 0:00:17 -       5 -   21.24 sent/s -   604.85 words/s - MLM-Bafia:  7.5391 || MLM-MKPAMAN_AMVOE_Ewondo:  8.6195 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  7.3385 -  - model LR: 1.0000e-04
INFO - 05/21/20 10:09:38 - 0:00:24 -      10 -   30.66 sent/s -  1775.10 words/s - MLM-Bafia:  5.8625 || MLM-MKPAMAN_AMVOE_Ewondo:  6.9503 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo: 

INFO - 05/21/20 10:15:03 - 0:05:50 - New best validation score: 315.734559
INFO - 05/21/20 10:15:03 - 0:05:50 - Saving checkpoint to ./dumped/mlm_tlm_BafiaEwondo/maml/checkpoint.pth ...
WARNING - 05/21/20 10:15:03 - 0:05:50 - Saving model parameters ...
WARNING - 05/21/20 10:15:03 - 0:05:50 - Saving model optimizer ...
INFO - 05/21/20 10:15:04 - 0:05:51 - ============ garbage collector collecting 0 ...
INFO - 05/21/20 10:15:04 - 0:05:51 - ============ Starting epoch 1 ... ============
INFO - 05/21/20 10:15:10 - 0:05:57 -     135 -   12.66 sent/s -  9913.03 words/s - MLM-Bafia:  4.8233 || MLM-MKPAMAN_AMVOE_Ewondo:  5.2253 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  4.8826 -  - model LR: 1.0000e-04
INFO - 05/21/20 10:15:10 - 0:05:57 - Creating new training data iterator (pred,Bafia) ...
INFO - 05/21/20 10:15:24 - 0:06:10 -     140 -   17.43 sent/s - 14159.05 words/s - MLM-Bafia:  4.9066 || MLM-MKPAMAN_AMVOE_Ewondo:  5.3673 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  5.1230 -  - model LR: 1.0000e-04
INFO

INFO - 05/21/20 10:20:45 - 0:11:31 - ============ garbage collector collecting 0 ...
INFO - 05/21/20 10:20:45 - 0:11:31 - ============ Starting epoch 2 ... ============
INFO - 05/21/20 10:20:54 - 0:11:40 -     270 -   11.91 sent/s - 18641.95 words/s - MLM-Bafia:  4.8733 || MLM-MKPAMAN_AMVOE_Ewondo:  5.4043 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  4.9212 -  - model LR: 1.0000e-04
INFO - 05/21/20 10:21:01 - 0:11:47 - Creating new training data iterator (pred,MKPAMAN_AMVOE_Ewondo) ...
INFO - 05/21/20 10:21:04 - 0:11:51 -     275 -   22.33 sent/s - 35632.91 words/s - MLM-Bafia:  4.8200 || MLM-MKPAMAN_AMVOE_Ewondo:  5.4532 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  4.8671 -  - model LR: 1.0000e-04
INFO - 05/21/20 10:21:11 - 0:11:57 - Creating new training data iterator (pred,Bafia) ...
INFO - 05/21/20 10:21:15 - 0:12:01 -     280 -   23.10 sent/s - 37495.59 words/s - MLM-Bafia:  4.7399 || MLM-MKPAMAN_AMVOE_Ewondo:  5.4162 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  4.6028 -  - model LR: 1.0000e-04
INFO - 05/21

...

INFO - 05/21/20 13:29:28 - 3:20:15 - ============ garbage collector collecting 0 ...
INFO - 05/21/20 13:29:28 - 3:20:15 - ============ Starting epoch 36 ... ============
INFO - 05/21/20 13:29:28 - 3:20:15 - Creating new training data iterator (pred,MKPAMAN_AMVOE_Ewondo) ...
INFO - 05/21/20 13:29:33 - 3:20:20 -    4790 -   10.00 sent/s - 277455.35 words/s - MLM-Bafia:  1.4900 || MLM-MKPAMAN_AMVOE_Ewondo:  1.2194 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  1.6671 -  - model LR: 1.0000e-04
INFO - 05/21/20 13:29:47 - 3:20:33 -    4795 -   17.32 sent/s - 480880.26 words/s - MLM-Bafia:  1.5533 || MLM-MKPAMAN_AMVOE_Ewondo:  1.1776 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  1.6545 -  - model LR: 1.0000e-04
INFO - 05/21/20 13:30:00 - 3:20:47 -    4800 -   18.11 sent/s - 503431.50 words/s - MLM-Bafia:  1.4906 || MLM-MKPAMAN_AMVOE_Ewondo:  1.1008 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  1.5880 -  - model LR: 1.0000e-04
INFO - 05/21/20 13:30:14 - 3:21:01 -    4805 -   17.23 sent/s - 481603.07 words/s - MLM-Bafia:  1.

INFO - 05/21/20 13:35:14 - 3:26:00 - ============ garbage collector collecting 0 ...
INFO - 05/21/20 13:35:14 - 3:26:00 - ============ Starting epoch 37 ... ============
INFO - 05/21/20 13:35:22 - 3:26:09 -    4925 -   11.78 sent/s - 335870.18 words/s - MLM-Bafia:  1.3758 || MLM-MKPAMAN_AMVOE_Ewondo:  1.1172 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  1.6902 -  - model LR: 1.0000e-04
INFO - 05/21/20 13:35:33 - 3:26:19 -    4930 -   22.36 sent/s - 638460.82 words/s - MLM-Bafia:  1.3608 || MLM-MKPAMAN_AMVOE_Ewondo:  1.1400 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  1.4438 -  - model LR: 1.0000e-04
INFO - 05/21/20 13:35:44 - 3:26:30 -    4935 -   22.36 sent/s - 639024.87 words/s - MLM-Bafia:  1.4853 || MLM-MKPAMAN_AMVOE_Ewondo:  1.2174 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  1.5050 -  - model LR: 1.0000e-04
INFO - 05/21/20 13:35:51 - 3:26:37 - Creating new training data iterator (pred,MKPAMAN_AMVOE_Ewondo) ...
INFO - 05/21/20 13:35:54 - 3:26:40 -    4940 -   23.05 sent/s - 659330.93 words/s - MLM-Bafia:  1.

INFO - 05/21/20 13:40:21 - 3:31:08 - ============ garbage collector collecting 0 ...
INFO - 05/21/20 13:40:21 - 3:31:08 - ============ Starting epoch 38 ... ============
INFO - 05/21/20 13:40:25 - 3:31:11 -    5055 -   10.12 sent/s - 296397.87 words/s - MLM-Bafia:  1.3485 || MLM-MKPAMAN_AMVOE_Ewondo:  1.0015 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  1.7521 -  - model LR: 1.0000e-04
INFO - 05/21/20 13:40:33 - 3:31:20 - Creating new training data iterator (pred,MKPAMAN_AMVOE_Ewondo) ...
INFO - 05/21/20 13:40:38 - 3:31:24 -    5060 -   18.52 sent/s - 544946.76 words/s - MLM-Bafia:  1.3911 || MLM-MKPAMAN_AMVOE_Ewondo:  1.1062 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  1.6755 -  - model LR: 1.0000e-04
INFO - 05/21/20 13:40:52 - 3:31:38 -    5065 -   17.03 sent/s - 499446.24 words/s - MLM-Bafia:  1.3848 || MLM-MKPAMAN_AMVOE_Ewondo:  1.0467 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  1.5525 -  - model LR: 1.0000e-04
INFO - 05/21/20 13:41:06 - 3:31:52 -    5070 -   17.11 sent/s - 502544.50 words/s - MLM-Bafia:  1.

INFO - 05/21/20 13:46:30 - 3:37:17 - ============ garbage collector collecting 0 ...
INFO - 05/21/20 13:46:30 - 3:37:17 - ============ Starting epoch 39 ... ============
INFO - 05/21/20 13:46:38 - 3:37:25 -    5190 -    9.81 sent/s - 294988.25 words/s - MLM-Bafia:  1.3927 || MLM-MKPAMAN_AMVOE_Ewondo:  1.0522 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  1.4754 -  - model LR: 1.0000e-04
INFO - 05/21/20 13:46:52 - 3:37:39 -    5195 -   17.54 sent/s - 527806.43 words/s - MLM-Bafia:  1.3374 || MLM-MKPAMAN_AMVOE_Ewondo:  1.0662 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  1.4994 -  - model LR: 1.0000e-04
INFO - 05/21/20 13:47:05 - 3:37:52 -    5200 -   18.15 sent/s - 546605.75 words/s - MLM-Bafia:  1.4269 || MLM-MKPAMAN_AMVOE_Ewondo:  1.0170 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  1.4697 -  - model LR: 1.0000e-04
INFO - 05/21/20 13:47:16 - 3:38:03 -    5205 -   21.96 sent/s - 661990.30 words/s - MLM-Bafia:  1.3093 || MLM-MKPAMAN_AMVOE_Ewondo:  0.9633 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  1.4978 -  - model LR: 1.000

INFO - 05/21/20 13:51:30 - 3:42:17 - ============ garbage collector collecting 0 ...
INFO - 05/21/20 13:51:30 - 3:42:17 - ============ Starting epoch 40 ... ============
INFO - 05/21/20 13:51:41 - 3:42:28 -    5325 -   11.99 sent/s - 369865.91 words/s - MLM-Bafia:  1.2742 || MLM-MKPAMAN_AMVOE_Ewondo:  0.9983 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  1.6335 -  - model LR: 1.0000e-04
INFO - 05/21/20 13:51:47 - 3:42:33 - Creating new training data iterator (pred,MKPAMAN_AMVOE_Ewondo) ...
INFO - 05/21/20 13:51:52 - 3:42:38 -    5330 -   22.74 sent/s - 701821.29 words/s - MLM-Bafia:  1.4586 || MLM-MKPAMAN_AMVOE_Ewondo:  0.9450 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  1.3613 -  - model LR: 1.0000e-04
INFO - 05/21/20 13:52:03 - 3:42:49 -    5335 -   21.83 sent/s - 674507.70 words/s - MLM-Bafia:  1.2902 || MLM-MKPAMAN_AMVOE_Ewondo:  0.9055 || MLM-Bafia-MKPAMAN_AMVOE_Ewondo:  1.5894 -  - model LR: 1.0000e-04
INFO - 05/21/20 13:52:11 - 3:42:58 -    5340 -   28.74 sent/s - 888849.23 words/s - MLM-Bafia:  1.

In [ ]:
# save the result
#! rsync -av --progress /content/meta_XLM/XLM/dumped/mlm_tlm_BafiaEwondo  /content/drive/"My Drive"/African_Translator/models/africa/cluster1

**Train a (unsupervised/supervised) MT from a pretrained meta-model**

In [21]:
%env epoch_size=6360
%env lgs=Bafia-MKPAMAN_AMVOE_Ewondo

env: epoch_size=6360
env: lgs=Bafia-MKPAMAN_AMVOE_Ewondo


In [22]:
# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

%env eval_bleu=true
! chmod +x src/evaluation/multi-bleu.perl

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: eval_bleu=true


In [15]:
%env stopping_criterion=valid_Bafia-MKPAMAN_AMVOE_Ewondo_mt_bleu,10
%env validation_metrics=valid_Bafia-MKPAMAN_AMVOE_Ewondo_mt_bleu
#%env reload_model=dumped/mlm_tlm_BafiaEwondo/maml/best-valid_mlm_ppl.pth,dumped/mlm_tlm_BafiaEwondo/maml/best-valid_mlm_ppl.pth
%env reload_model=dumped/mlm_tlm_BafiaEwondo/maml/best-valid_mlm_ppl.pth,dumped/mlm_tlm_BafiaEwondo/maml/best-valid_mlm_ppl.pth
%env ae_steps=Bafia,MKPAMAN_AMVOE_Ewondo
%env bt_steps=Bafia-MKPAMAN_AMVOE_Ewondo-Bafia,MKPAMAN_AMVOE_Ewondo-Bafia-MKPAMAN_AMVOE_Ewondo

env: stopping_criterion=valid_Bafia-MKPAMAN_AMVOE_Ewondo_mt_bleu,10
env: validation_metrics=valid_Bafia-MKPAMAN_AMVOE_Ewondo_mt_bleu
env: reload_model=dumped/mlm_tlm_BafiaEwondo/maml/best-valid_mlm_ppl.pth,dumped/mlm_tlm_BafiaEwondo/maml/best-valid_mlm_ppl.pth
env: ae_steps=Bafia,MKPAMAN_AMVOE_Ewondo
env: bt_steps=Bafia-MKPAMAN_AMVOE_Ewondo-Bafia,MKPAMAN_AMVOE_Ewondo-Bafia-MKPAMAN_AMVOE_Ewondo


In [ ]:
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1

%env batch_size=4
%env max_epoch=100

# unsupervised MT
#! python train.py --exp_name UnSupMT_BafiaEwondo --exp_id $exp_id --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

In [ ]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/UnSupMT_BafiaEwondo  /content/drive/"My Drive"/African_Translator/models/africa/cluster1

In [17]:
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1

%env batch_size=4
%env max_epoch=100
dump_path=./dumped/
# supervised MT
%env mt_steps=Bafia-MKPAMAN_AMVOE_Ewondo,MKPAMAN_AMVOE_Ewondo-Bafia          
! python train.py --exp_name SupMT_BafiaEwondo --exp_id $exp_id  --dump_path $dump_path --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --mt_steps $mt_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: batch_size=4
env: max_epoch=100
env: mt_steps=Bafia-MKPAMAN_AMVOE_Ewondo,MKPAMAN_AMVOE_Ewondo-Bafia
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : 0ef8c54241e8
INFO - 05/19/20 15:37:08 - 0:00:00 - ============ Initialized logger ============
INFO - 05/19/20 15:37:08 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: ['Bafia', 'MKPAMAN_AMVOE_Ewondo']
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 4
              

KeyboardInterrupt: ignored

In [20]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/SupMT_BafiaEwondo  /content/drive/"My Drive"/African_Translator/models/africa/cluster1